In [113]:
import re
import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader

# Cargar texto
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

raw_text = raw_text.replace("\n", " ").strip()
print(raw_text[:300])
print("\nChars:", len(raw_text))

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would ha

Chars: 20479


In [114]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

print("N tokens (simple regex):", len(preprocessed))
print(preprocessed[:30])

N tokens (simple regex): 4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [115]:
all_words = sorted(set(preprocessed))
vocab = {token: integer for integer, token in enumerate(all_words)}

# Tokens especiales (clave)
special_tokens = ["<|unk|>", "<|endoftext|>"]
for tok in special_tokens:
    if tok not in vocab:
        vocab[tok] = len(vocab)

print("Vocab size:", len(vocab))
print("IDs especiales:", vocab["<|unk|>"], vocab["<|endoftext|>"])

Vocab size: 1132
IDs especiales: 1130 1131


In [116]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int.get(s, self.str_to_int["<|unk|>"]) for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

tokenizer_simple = SimpleTokenizerV1(vocab)

test_text = "Hello, do you like tea. Is this-- a test?"
encoded = tokenizer_simple.encode(test_text)
decoded = tokenizer_simple.decode(encoded)

print("Encoded:", encoded[:20])
print("Decoded:", decoded)

Encoded: [1130, 5, 355, 1126, 628, 975, 7, 1130, 999, 6, 115, 1130, 10]
Decoded: <|unk|>, do you like tea. <|unk|> this -- a <|unk|>?


In [117]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Need at least max_length+1 tokens"

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128,
                         shuffle=True, drop_last=True, num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [118]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

inputs, targets = next(iter(dataloader))
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)
print("\nShapes:", inputs.shape, targets.shape)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])

Shapes: torch.Size([8, 4]) torch.Size([8, 4])


In [119]:
tokenizer = tiktoken.get_encoding("gpt2")

def n_samples(txt, max_length, stride):
    ds = GPTDatasetV1(txt, tokenizer, max_length=max_length, stride=stride)
    return len(ds)

configs = [
    (16, 16),  # sin superposición
    (16, 8),   # 50% overlap
    (32, 8),   # más contexto + overlap alto
]

for ml, st in configs:
    print(f"max_length={ml}, stride={st} -> muestras = {n_samples(raw_text, ml, st)}")

max_length=16, stride=16 -> muestras = 316
max_length=16, stride=8 -> muestras = 631
max_length=32, stride=8 -> muestras = 629


In [120]:
vocab_size = 50257      # GPT-2 vocab
embed_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, embed_dim)

# Tomar un batch pequeño
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
inputs, targets = next(iter(dataloader))

token_embeddings = token_embedding_layer(inputs)
print("inputs shape:", inputs.shape)
print("token_embeddings shape:", token_embeddings.shape)  # (batch, seq_len, embed_dim)

inputs shape: torch.Size([8, 4])
token_embeddings shape: torch.Size([8, 4, 256])


In [121]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, embed_dim)

positions = torch.arange(context_length)
pos_embeddings = pos_embedding_layer(positions)

print("positions:", positions)
print("pos_embeddings shape:", pos_embeddings.shape)  # (seq_len, embed_dim)

# Combinar token + posición
x = token_embeddings + pos_embeddings
print("x shape:", x.shape)

positions: tensor([0, 1, 2, 3])
pos_embeddings shape: torch.Size([4, 256])
x shape: torch.Size([8, 4, 256])


# Tokenización: pasar de texto a unidades que podamos manejar

La tokenización es el primer paso para poder trabajar con lenguaje natural en un modelo. Básicamente nos convierte el texto en pequeñas unidades llamadas tokens, que luego se van a transforman en números (IDs). Esto es importante porque las redes neuronales no entienden palabras directamente, sino que entienden números.

Sin tokenización no podríamos entrenar un modelo de lenguaje, ya que no habría una forma estructurada de representar el texto. En el contexto de agentes, es importante: que el agente necesita representar instrucciones, contexto, memoria y respuestas de forma consistente para poder razonar y generar acciones que tengan sentido. En pocas palabras, tokenizar es convertir lenguaje humano en algo que una red neuronal pueda procesar matemáticamente.

# Vocabulario + <|unk|> y robustez

Cuando estamos construyendo un vocabulario, asignamos un ID único a cada token la cual aparece en el corpus. Esto permite que cada palabra o subpalabra tenga una representación numérica asignada fija dentro del modelo, aunque en el mundo real siempre aparecen palabras nuevas: nombres propios, errores de escritura, términos técnicos, entre otros.

Aquí es donde vemos el uso de el token <|unk|>. Este token funciona como respaldo cuando aparece algo que no está en el vocabulario. En vez de que el sistema falle, el modelo asigna ese token desconocido y sigue funcionando. Esto aporta robustez y confianza al pipeline.

En sistemas de agentes esto es fundamental, porque los agentes reciben entradas impredecibles (usuarios, herramientas externas, logs, datos dinámicos). Si no existiera un mecanismo como <|unk|>, el sistema podría romperse fácilmente ante cualquier palabra nueva.

# Ventanas (max_length/stride) y overlap

Los modelos de lenguaje se entrenan para predecir el siguiente token. Para poder lograrlo, el texto completo se divide en secuencias de longitud fija max_length, luego se recorren usando un paso stride. 

Si el stride es menor que max_length, se va a genera superposición entre las ventanas. Esto va a significar que algunos tokens aparecen en múltiples secuencias de entrenamiento, pero en contextos la cual son muy poco distintos. Esta técnica ayuda al modelo a aprender mejor las dependencias locales y reduce los huecos entre fragmentos de texto.

en pocas palabras, el overlap funciona como una forma de aumentar los datos sin necesidad de recolectar más texto. Le da al modelo más oportunidades de ver patrones similares en distintos contextos, lo cual mejora su capacidad de generalización.

# ¿Por qué los embeddings codifican significado? Relación con redes neuronales

Los embeddings lo que hacen es que codifican significado porque son vectores que se aprenden durante el entrenamiento del modelo. No son representaciones aleatorias, se estan optimizando, usando gradiente para resolver una tarea, como predecir el siguiente token. 

Cuando dos palabras aparecen en contextos similares, el modelo ajusta sus vectores para que produzcan activaciones parecidas en la red. Como resultado, esos vectores terminan ubicándose cerca en el espacio vectorial.

Desde el punto de vista de las redes neuronales, el embedding es una capa con pesos entrenables (una tabla de vectores) que transforma IDs discretos en representaciones continuas. Estas representaciones luego pasan por más capas que capturan patrones más complejos y relaciones de largo alcance.

En sistemas de agentes, los embeddings son esenciales porque lo que hacen es permitir comparar información, hacer búsqueda semántica, recuperar memoria y medir similitud entre conceptos.

## Resultado del experimento (max_length/stride)

Al bajar el stride aumentan las muestras porque se generan más ventanas superpuestas del mismo texto. Eso hace que el modelo vea los mismos tokens en distintos contextos, lo cual nos ayuda a que pueda aprender mejor dependencias locales y a no perder transiciones entre ventanas. Por eso el overlap funciona como una forma de aumentar datos sin traer más texto.